In [1]:
import pathlib

import pyarrow.dataset as ds
from datafusion import SessionContext, col, lit
from deltalake import DeltaTable

# DataFusion Read Delta

In [2]:
ctx = SessionContext()

In [3]:
table = DeltaTable(f"{pathlib.Path.home()}/data/delta/G1_1e8_1e2_0_0")

### With PyArrow dataset

In [5]:
%%time
ctx.register_dataset("my_dataset", table.to_pyarrow_dataset())

CPU times: user 3.82 ms, sys: 4.53 ms, total: 8.34 ms
Wall time: 8.49 ms


In [6]:
%%time
ctx.sql("select * from my_dataset where v2 > 5")

CPU times: user 1.36 ms, sys: 5.43 ms, total: 6.79 ms
Wall time: 9.67 ms


DataFrame()
+-------+-------+--------------+-----+-----+--------+----+----+-----------+
| id1   | id2   | id3          | id4 | id5 | id6    | v1 | v2 | v3        |
+-------+-------+--------------+-----+-----+--------+----+----+-----------+
| id082 | id049 | id0000022715 | 97  | 55  | 756924 | 2  | 11 | 74.161136 |
| id053 | id052 | id0000113549 | 19  | 56  | 139048 | 1  | 10 | 95.178444 |
| id090 | id043 | id0000637409 | 94  | 50  | 12448  | 3  | 12 | 60.21896  |
| id071 | id092 | id0000537978 | 52  | 51  | 638154 | 2  | 11 | 41.181681 |
| id099 | id017 | id0000324904 | 100 | 98  | 650130 | 5  | 6  | 53.967191 |
| id068 | id010 | id0000645245 | 85  | 15  | 392202 | 5  | 15 | 47.74931  |
| id080 | id005 | id0000722692 | 54  | 90  | 737105 | 4  | 9  | 37.710627 |
| id060 | id084 | id0000269745 | 70  | 93  | 496965 | 2  | 10 | 45.513606 |
| id085 | id004 | id0000020734 | 65  | 10  | 861078 | 3  | 9  | 35.692509 |
| id068 | id095 | id0000888093 | 37  | 3   | 395630 | 5  | 11 | 11.12151  |


In [11]:
ctx.register_dataset("some_table", table.to_pyarrow_dataset())

In [12]:
ctx.sql("select * from some_table where v2 > 5")

DataFrame()
+-------+-------+--------------+-----+-----+--------+----+----+-----------+
| id1   | id2   | id3          | id4 | id5 | id6    | v1 | v2 | v3        |
+-------+-------+--------------+-----+-----+--------+----+----+-----------+
| id007 | id096 | id0000445327 | 76  | 60  | 566466 | 2  | 11 | 82.691753 |
| id018 | id078 | id0000576456 | 52  | 92  | 193437 | 3  | 8  | 32.02124  |
| id019 | id049 | id0000790237 | 11  | 85  | 979819 | 4  | 13 | 17.001827 |
| id052 | id058 | id0000041914 | 42  | 12  | 198611 | 5  | 13 | 66.46749  |
| id010 | id016 | id0000921752 | 11  | 99  | 166079 | 1  | 7  | 61.637281 |
| id033 | id027 | id0000054357 | 100 | 79  | 793235 | 1  | 13 | 65.844356 |
| id049 | id031 | id0000237885 | 21  | 96  | 849387 | 3  | 7  | 40.847662 |
| id036 | id100 | id0000840597 | 26  | 92  | 51226  | 2  | 8  | 38.039983 |
| id010 | id033 | id0000128874 | 33  | 18  | 493856 | 3  | 15 | 91.906635 |
| id026 | id045 | id0000045901 | 35  | 94  | 785336 | 4  | 13 | 11.126579 |


### With PyArrow table

In [7]:
ctx.register_table("my_table", table.to_pyarrow_table())

TypeError: argument 'table': 'Table' object cannot be converted to 'Table'

### With batches

In [8]:
%%time
df = ctx.create_dataframe([table.to_pyarrow_table().to_batches()])

CPU times: user 4.99 s, sys: 1.14 s, total: 6.13 s
Wall time: 1.28 s


In [9]:
%%time
df.filter(col("v2") > lit(5))

CPU times: user 144 µs, sys: 325 µs, total: 469 µs
Wall time: 1.83 ms


DataFrame()
+-------+-------+--------------+-----+-----+--------+----+----+-----------+
| id1   | id2   | id3          | id4 | id5 | id6    | v1 | v2 | v3        |
+-------+-------+--------------+-----+-----+--------+----+----+-----------+
| id007 | id096 | id0000445327 | 76  | 60  | 566466 | 2  | 11 | 82.691753 |
| id018 | id078 | id0000576456 | 52  | 92  | 193437 | 3  | 8  | 32.02124  |
| id019 | id049 | id0000790237 | 11  | 85  | 979819 | 4  | 13 | 17.001827 |
| id052 | id058 | id0000041914 | 42  | 12  | 198611 | 5  | 13 | 66.46749  |
| id010 | id016 | id0000921752 | 11  | 99  | 166079 | 1  | 7  | 61.637281 |
| id033 | id027 | id0000054357 | 100 | 79  | 793235 | 1  | 13 | 65.844356 |
| id049 | id031 | id0000237885 | 21  | 96  | 849387 | 3  | 7  | 40.847662 |
| id036 | id100 | id0000840597 | 26  | 92  | 51226  | 2  | 8  | 38.039983 |
| id010 | id033 | id0000128874 | 33  | 18  | 493856 | 3  | 15 | 91.906635 |
| id026 | id045 | id0000045901 | 35  | 94  | 785336 | 4  | 13 | 11.126579 |


## Delta Lake vs Parquet

In [14]:
table = DeltaTable(f"{pathlib.Path.home()}/data/deltalake/G1_1e9_1e2_0_0")

In [15]:
%%time
ctx.register_dataset("my_delta_table", table.to_pyarrow_dataset())

Exception: DataFusion error: Execution("The table my_delta_table already exists")

In [30]:
%%time
res = ctx.sql("select id1, sum(v1) as v1 from my_delta_table where id1='id096' group by id1")

CPU times: user 635 µs, sys: 560 µs, total: 1.2 ms
Wall time: 587 µs


In [31]:
%%time
print(res)

DataFrame()
+-------+----------+
| id1   | v1       |
+-------+----------+
| id096 | 29993372 |
+-------+----------+
CPU times: user 14.8 s, sys: 3.45 s, total: 18.2 s
Wall time: 2.81 s


In [18]:
%%time
res = ctx.sql("select id1, sum(v1) as v1 from my_delta_table group by id1")

CPU times: user 803 µs, sys: 246 µs, total: 1.05 ms
Wall time: 583 µs


In [19]:
%%time
print(res)

DataFrame()
+-------+----------+
| id1   | v1       |
+-------+----------+
| id016 | 30003304 |
| id014 | 29998476 |
| id020 | 29993667 |
| id075 | 30013372 |
| id096 | 29993372 |
| id010 | 30006196 |
| id055 | 30009993 |
| id095 | 30010887 |
| id086 | 30003608 |
| id033 | 29983262 |
+-------+----------+
CPU times: user 55.6 s, sys: 6.34 s, total: 1min 1s
Wall time: 8.67 s


### Run the same query with Parquet

In [10]:
path = f"{pathlib.Path.home()}/data/G1_1e9_1e2_0_0.parquet"

In [11]:
%%time
ctx.register_parquet("my_parquet_table", path)

CPU times: user 298 ms, sys: 55.5 ms, total: 354 ms
Wall time: 358 ms


In [28]:
%%time
res = ctx.sql("select id1, sum(v1) as v1 from my_parquet_table where id1='id096' group by id1")

CPU times: user 882 µs, sys: 182 µs, total: 1.06 ms
Wall time: 593 µs


In [29]:
%%time
print(res)

DataFrame()
+-------+----------+
| id1   | v1       |
+-------+----------+
| id096 | 29993372 |
+-------+----------+
CPU times: user 17.9 s, sys: 6.25 s, total: 24.1 s
Wall time: 5.29 s


In [20]:
%%time
res = ctx.sql("select id1, sum(v1) as v1 from my_parquet_table group by id1")

CPU times: user 520 µs, sys: 103 µs, total: 623 µs
Wall time: 335 µs


In [21]:
%%time
print(res)

DataFrame()
+-------+----------+
| id1   | v1       |
+-------+----------+
| id047 | 29972409 |
| id091 | 29995955 |
| id078 | 29998434 |
| id052 | 30014118 |
| id026 | 29993858 |
| id048 | 29998849 |
| id034 | 30010786 |
| id006 | 30006882 |
| id066 | 29997843 |
| id084 | 30005578 |
+-------+----------+
CPU times: user 37.3 s, sys: 5.69 s, total: 43 s
Wall time: 7.37 s
